# Lab

Using the following code as a basis, use async methods to fetch the temperatures in the given cities:

In [1]:
APPID = '10d4440bbaa8581bb8da9bd1fbea5617'
UNITS = 'metric'
# UNITS = 'imperial'
cities = [
    'Boulder', 'Atlanta', 'San Francisco',
    'Reno', 'Honolulu', 'Zurich', 'Dubai',
    'Dublin','Stuttgart', 'Rome', 'Singapore', 
    'Bangalore', 'Hyderabad', 'Seattle',
]

from aiohttp_requests import requests

async def get_temp(city, units=UNITS, appid=APPID):
    url = 'http://api.openweathermap.org/data/2.5/weather'
    resp = await requests.get(url, params={
        'q': city,
        'units': units,
        'appid': appid,        
    })
    resp.raise_for_status()
    json_data = await resp.json()
    temp = json_data['main']['temp']
    return temp

In [2]:
import asyncio

coroutines = [get_temp(city) for city in cities]

coroutines

[<coroutine object get_temp at 0x7fe3fce63340>,
 <coroutine object get_temp at 0x7fe3fce633c0>,
 <coroutine object get_temp at 0x7fe3fce63440>,
 <coroutine object get_temp at 0x7fe3fce634c0>,
 <coroutine object get_temp at 0x7fe3fce63540>,
 <coroutine object get_temp at 0x7fe3fce635c0>,
 <coroutine object get_temp at 0x7fe3fce63640>,
 <coroutine object get_temp at 0x7fe3fce636c0>,
 <coroutine object get_temp at 0x7fe3fce63740>,
 <coroutine object get_temp at 0x7fe3fce63840>,
 <coroutine object get_temp at 0x7fe3fce638c0>,
 <coroutine object get_temp at 0x7fe3fce63940>,
 <coroutine object get_temp at 0x7fe3fce639c0>,
 <coroutine object get_temp at 0x7fe3fce63a40>]

In [3]:
result = await asyncio.gather(*coroutines)
# result = await asyncio.gather(coroutines[0], coroutines[1], ...)

See also multiprocessing.Pool.[map, imap, imap_unordered]

In [4]:
result

[26.26,
 29.98,
 21.84,
 15.14,
 28.55,
 9.71,
 32.54,
 22.51,
 9.48,
 29.78,
 25.98,
 18.9,
 23.73,
 23.36]

In [5]:
cities

['Boulder',
 'Atlanta',
 'San Francisco',
 'Reno',
 'Honolulu',
 'Zurich',
 'Dubai',
 'Dublin',
 'Stuttgart',
 'Rome',
 'Singapore',
 'Bangalore',
 'Hyderabad',
 'Seattle']

In [6]:
dict(zip(cities, result))

{'Boulder': 26.26,
 'Atlanta': 29.98,
 'San Francisco': 21.84,
 'Reno': 15.14,
 'Honolulu': 28.55,
 'Zurich': 9.71,
 'Dubai': 32.54,
 'Dublin': 22.51,
 'Stuttgart': 9.48,
 'Rome': 29.78,
 'Singapore': 25.98,
 'Bangalore': 18.9,
 'Hyderabad': 23.73,
 'Seattle': 23.36}

In [7]:
async def serialized():
    result = []
    for city in cities:
        result.append(await get_temp(city))
    return result

In [8]:
result = await serialized()

In [9]:
result

[26.26,
 29.81,
 21.84,
 15.18,
 28.55,
 9.71,
 32.54,
 22.5,
 9.48,
 29.81,
 25.98,
 18.9,
 23.73,
 23.46]

How about queues?

In [10]:
from asyncio import Queue

In [11]:
q = Queue()

async def worker(i, q):
    while True:
        city = await q.get()
        temp = await get_temp(city)
        print(i, city, temp)

In [12]:
for city in cities:
    await q.put(city)

In [13]:
workers = [worker(i, q) for i in range(4)]

In [14]:
loop = asyncio.get_event_loop()
for w in workers:
    loop.create_task(w)

0 Boulder 26.26
1 Atlanta 29.98
2 San Francisco 21.84
3 Reno 15.18
0 Honolulu 28.55
1 Zurich 9.71
2 Dubai 32.54
0 Stuttgart 9.48
3 Dublin 22.5
1 Rome 29.81
0 Bangalore 18.9
2 Singapore 25.98
3 Hyderabad 23.73
1 Seattle 23.36
